In [203]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import pandas as pd
import random

### MongoDB

In [204]:
uri = 'mongodb://localhost/'
db = MongoClient(uri)['vendas']

### estoque

In [205]:
xls = pd.read_excel('estoque.xlsx')

In [206]:
estoques = []
for i, e in enumerate(xls.to_dict('records')):
  estoques.append({
    'id': i + 1,
    'produto': {
      'marca': e['marca'],
      'categoria': e['categoria'],
      'descricao': e['descricao'],
    },
    'quantidade': e['quantidade'],
    'preco_compra': e['preco_compra'] * (random.random() * 0.5 + 0.2),
    'preco_venda': e['preco_compra'],
  })
estoques

[{'id': 1,
  'produto': {'marca': 'KazeKami',
   'categoria': 'Secadores',
   'descricao': 'Secador de Parede 1200W'},
  'quantidade': 1000,
  'preco_compra': 79.25548255817442,
  'preco_venda': 149.73},
 {'id': 2,
  'produto': {'marca': 'KazeKami',
   'categoria': 'Secadores',
   'descricao': 'Secador de Parede 1200W'},
  'quantidade': 1000,
  'preco_compra': 73.9690008760712,
  'preco_venda': 169.9},
 {'id': 3,
  'produto': {'marca': 'KazeKami',
   'categoria': 'Secadores',
   'descricao': 'Secador de Parede 1400W'},
  'quantidade': 1000,
  'preco_compra': 295.4559744929262,
  'preco_venda': 460.35},
 {'id': 4,
  'produto': {'marca': 'KazeKami',
   'categoria': 'Secadores',
   'descricao': 'Secador de Parede 1400W (Preto)'},
  'quantidade': 1000,
  'preco_compra': 122.49997960731771,
  'preco_venda': 469.26},
 {'id': 5,
  'produto': {'marca': 'KazeKami',
   'categoria': 'Secadores',
   'descricao': 'Secador de Parede 1500W'},
  'quantidade': 1000,
  'preco_compra': 233.7068363685849,

In [207]:
db['venda_estoque'].delete_many({})
db['venda_estoque'].insert_many(estoques)

InsertManyResult([ObjectId('66449990ee56e6635d192f00'), ObjectId('66449990ee56e6635d192f01'), ObjectId('66449990ee56e6635d192f02'), ObjectId('66449990ee56e6635d192f03'), ObjectId('66449990ee56e6635d192f04'), ObjectId('66449990ee56e6635d192f05'), ObjectId('66449990ee56e6635d192f06'), ObjectId('66449990ee56e6635d192f07'), ObjectId('66449990ee56e6635d192f08'), ObjectId('66449990ee56e6635d192f09'), ObjectId('66449990ee56e6635d192f0a'), ObjectId('66449990ee56e6635d192f0b'), ObjectId('66449990ee56e6635d192f0c'), ObjectId('66449990ee56e6635d192f0d'), ObjectId('66449990ee56e6635d192f0e'), ObjectId('66449990ee56e6635d192f0f'), ObjectId('66449990ee56e6635d192f10'), ObjectId('66449990ee56e6635d192f11'), ObjectId('66449990ee56e6635d192f12'), ObjectId('66449990ee56e6635d192f13'), ObjectId('66449990ee56e6635d192f14'), ObjectId('66449990ee56e6635d192f15'), ObjectId('66449990ee56e6635d192f16'), ObjectId('66449990ee56e6635d192f17'), ObjectId('66449990ee56e6635d192f18'), ObjectId('66449990ee56e6635d192f

# Pedidos

### base original

In [208]:
df_ori = pd.read_csv('Sales_Product_Combined.csv')
df_ori

,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,176558,USB-C Charging Cable,2,11.95,19-04-2019,8:46 AM,"917 1st St, Dallas, TX 75001",Dallas,Cable
1,176559,Bose SoundSport Headphones,1,99.99,07-04-2019,10:30 PM,"682 Chestnut St, Boston, MA 02215",Boston,Headphones
2,176560,Google Phone,1,600,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Phone
3,176560,Wired Headphones,1,11.99,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Headphones
4,176561,Wired Headphones,1,11.99,30-04-2019,9:27 AM,"333 8th St, Los Angeles, CA 90001",Los Angeles,Headphones
...,...,...,...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2.99,17-09-2019,8:56 PM,"840 Highland St, Los Angeles, CA 90001",Los Angeles,Batteries
185946,259354,iPhone,1,700,01-09-2019,4:00 PM,"216 Dogwood St, San Francisco, CA 94016",San Francisco,iPhone
185947,259355,iPhone,1,700,23-09-2019,7:39 AM,"220 12th St, San Francisco, CA 94016",San Francisco,iPhone
185948,259356,34in Ultrawide Monitor,1,379.99,19-09-2019,5:30 PM,"511 Forest St, San Francisco, CA 94016",San Francisco,Monitor


In [209]:
# quantidade por cidade
df_ori.groupby('City')['Quantity Ordered'].count().sort_values()

City
Austin            9905
Portland         12465
Seattle          14732
Dallas           14820
Atlanta          14881
Boston           19934
New York City    24876
Los Angeles      29605
San Francisco    44732
Name: Quantity Ordered, dtype: int64

In [210]:
df_ori.groupby('Product')['Quantity Ordered'].count().sort_values()

Product
LG Dryer                        646
LG Washing Machine              666
Vareebadd Phone                2065
20in Monitor                   4101
ThinkPad Laptop                4128
Macbook Pro Laptop             4724
Flatscreen TV                  4800
Google Phone                   5525
34in Ultrawide Monitor         6181
27in 4K Gaming Monitor         6230
iPhone                         6842
27in FHD Monitor               7507
Bose SoundSport Headphones    13325
Apple Airpods Headphones      15549
Wired Headphones              18882
AA Batteries (4-pack)         20577
AAA Batteries (4-pack)        20641
Lightning Charging Cable      21658
USB-C Charging Cable          21903
Name: Quantity Ordered, dtype: int64

In [211]:
df_ori.groupby('Product')['Quantity Ordered'].sum().sort_values()

Product
LG Dryer                        646
LG Washing Machine              666
Vareebadd Phone                2068
20in Monitor                   4129
ThinkPad Laptop                4130
Macbook Pro Laptop             4728
Flatscreen TV                  4819
Google Phone                   5532
34in Ultrawide Monitor         6199
27in 4K Gaming Monitor         6244
iPhone                         6849
27in FHD Monitor               7550
Bose SoundSport Headphones    13457
Apple Airpods Headphones      15661
Wired Headphones              20557
Lightning Charging Cable      23217
USB-C Charging Cable          23975
AA Batteries (4-pack)         27635
AAA Batteries (4-pack)        31017
Name: Quantity Ordered, dtype: int64

In [212]:
df_ori.query('City == " Austin"').groupby('Product')['Quantity Ordered'].sum().sort_values()

Product
LG Washing Machine              26
LG Dryer                        55
Vareebadd Phone                108
ThinkPad Laptop                210
20in Monitor                   230
Flatscreen TV                  242
Macbook Pro Laptop             251
Google Phone                   274
27in 4K Gaming Monitor         318
34in Ultrawide Monitor         328
27in FHD Monitor               360
iPhone                         377
Bose SoundSport Headphones     707
Apple Airpods Headphones       887
Wired Headphones              1130
USB-C Charging Cable          1251
Lightning Charging Cable      1307
AA Batteries (4-pack)         1424
AAA Batteries (4-pack)        1668
Name: Quantity Ordered, dtype: int64

### quantidades: Austin=9905, Boston=19934, San Francisco=44732

In [213]:
# renomeando campos
df_aux = df_ori.rename(columns={
  'Order ID': 'id', 
  'Product' : 'produto', 
  'Quantity Ordered': 'quantidade', 
  'Price': 'preco', 
  'Order Date': 'data',
  'Time': 'hora',
  'Product Type': 'categoria', 
  'Purchase Address': 'endereco', 
  'City': 'cidade',
})

# criando campo horário
df_aux['horario'] = pd.to_datetime(df_aux['data'] + ' ' + df_aux['hora'], format='%d-%m-%Y %I:%M %p')

# diminuindo quantidade de registros
df_aux['cidade'] = df_aux['cidade'].str.strip()
# df_aux = df_aux.query('horario.dt.year == 2019 and cidade == "San Francisco"')[
df_aux = df_aux.query('horario.dt.year == 2019 and cidade == "Austin"')[
  ['horario', 'produto', 'quantidade', 'preco']
]

### ajustando os valores

In [214]:
df = df_aux.sort_values('horario')

# campo auxiliar para aleatórios
df['rnd'] = [random.randrange(100) for _ in df.index]

# transferir 25% das vendas dos meses 3, 4 e 5 para os meses 9, 10 e 11
df['horario'] = df['horario'].mask(
  (df['horario'] > pd.to_datetime('01-03-2019', format='%d-%m-%Y')) & 
  (df['horario'] < pd.to_datetime('01-06-2019', format='%d-%m-%Y')) & 
  (df['rnd'] < 25), 
  df['horario'] + pd.DateOffset(months=6))

# transferir 20% das vendas do mês 12 para o mês 1
df['horario'] = df['horario'].mask(
  (df['horario'] > pd.to_datetime('01-12-2019', format='%d-%m-%Y')) & 
  (df['rnd'] < 20), 
  df['horario'] + pd.DateOffset(months=-11))

# distribuir aleatóriamente os registros em 5 anos
df['rnd'] = [random.randrange(100) for _ in df.index]
df['horario'] = df['horario'].mask(
  (df.rnd < 25), 
  df['horario'] + pd.DateOffset(years=2))
df['horario'] = df['horario'].mask(
  (df.rnd >= 25) & 
  (df.rnd < 50),
  df['horario'] + pd.DateOffset(years=3))
df['horario'] = df['horario'].mask(
  (df.rnd >= 50) & 
  (df.rnd < 75),
  df['horario'] + pd.DateOffset(years=4))
df['horario'] = df['horario'].mask(
  (df.rnd >= 75),
  df['horario'] + pd.DateOffset(years=5))

# campo auxiliar de data
df['data'] = pd.to_datetime(df['horario']).dt.date

# distribuir os itens do pedido
df['nota'] = df['horario'].dt.strftime('%Y%m%d-%M').str.slice(stop=10)

# permitir cálculo com o preço
df['preco'] = df['preco'].str.replace(',', '').astype(float)
df['preco'] = df.apply(lambda row: row['preco'] * 10 if row['preco'] < 80 else row['preco'], axis=1)
df['preco'] = df['preco'].round(2)

# calcular aleatóriamente quantidade, mas valores menores tem peso maior
pop = [i + 1 for i in range(40)]
wei = [i * i + 1 for i in range(40, 0, -1)]
df['quantidade'] = random.choices(pop, wei, k=len(df.index))
df['quantidade'] = (df['quantidade'] / (df['quantidade'] * df['preco'] / 10000 + 1).astype(int)).astype(int)
df['total'] = df['quantidade'] * df['preco']
df['total'] = df['total'].round(2)

# renomear produtos
prod = {
  'LG Dryer': '',
  'LG Washing Machine': '',
  'Vareebadd Phone': '',
  '20in Monitor': '',
  'ThinkPad Laptop': '',
  'Macbook Pro Laptop': '',
  'Flatscreen TV': '',
  'Google Phone': '',
  '34in Ultrawide Monitor': 'P29',
  '27in 4K Gaming Monitor': 'P28',
  'iPhone': 'P27',
  '27in FHD Monitor': 'P26',
  'Bose SoundSport Headphones': 'P25',
  'Apple Airpods Headphones': 'P24',
  'Wired Headphones': 'P23',
  'Lightning Charging Cable': 'P22',
  'USB-C Charging Cable': 'P21',
  'AA Batteries (4-pack)': 'P20',
  'AAA Batteries (4-pack)': 'P19',
}

df['produto'] = df.apply(
  lambda row: 
    prod[row['produto']] if row['produto'] in [
      # 'LG Dryer', 
      # 'LG Washing Machine', 
      # 'Vareebadd Phone', 
      # '20in Monitor', 
      # 'ThinkPad Laptop', 
      # 'Macbook Pro Laptop', 
      # 'Flatscreen TV', 
      # 'Google Phone', 

      '34in Ultrawide Monitor',
      '27in 4K Gaming Monitor',
      'iPhone',

      '27in FHD Monitor',
      'Bose SoundSport Headphones',
      'Apple Airpods Headphones',
      'Wired Headphones',
      'Lightning Charging Cable',
      'USB-C Charging Cable',
      'AA Batteries (4-pack)',
      'AAA Batteries (4-pack)'
    ] else f'P{random.randrange(1, 19):02}', axis=1)

df = df[['nota', 'data', 'horario', 'produto', 'quantidade']].sort_values('horario')
df

,nota,data,horario,produto,quantidade
32591,20210101-3,2021-01-01,2021-01-01 10:31:00,P23,14
69596,20210101-3,2021-01-01,2021-01-01 15:39:00,P25,3
72375,20210102-1,2021-01-02,2021-01-02 00:10:00,P24,9
76715,20210102-1,2021-01-02,2021-01-02 09:16:00,P15,13
73081,20210102-3,2021-01-02,2021-01-02 11:30:00,P19,2
...,...,...,...,...,...
37685,20241231-4,2024-12-31,2024-12-31 08:45:00,P20,7
32248,20241231-3,2024-12-31,2024-12-31 09:31:00,P21,5
53496,20241231-0,2024-12-31,2024-12-31 12:09:00,P20,5
43567,20241231-0,2024-12-31,2024-12-31 17:02:00,P26,5


In [215]:
# df.head()
# df.info()
# df[:20]
# df[-20:]
df.shape
# df_aux.shape
# df_aux[:20]
# df_aux[-20:]

(9903, 5)

In [216]:
# quantidade por mês
j = df[['horario', 'quantidade']]
j.index = df.horario
j.resample(rule='ME', on='horario')['quantidade'].sum()

horario
2021-01-31    2113
2021-02-28    1544
2021-03-31    1788
2021-04-30    1863
2021-05-31    1674
2021-06-30    1801
2021-07-31    2025
2021-08-31    1823
2021-09-30    2216
2021-10-31    3114
2021-11-30    2749
2021-12-31    2724
2022-01-31    2034
2022-02-28    1364
2022-03-31    1542
2022-04-30    2013
2022-05-31    1420
2022-06-30    1563
2022-07-31    1749
2022-08-31    1501
2022-09-30    2173
2022-10-31    3458
2022-11-30    2529
2022-12-31    2826
2023-01-31    1930
2023-02-28    1984
2023-03-31    1839
2023-04-30    1771
2023-05-31    1492
2023-06-30    1789
2023-07-31    1906
2023-08-31    1566
2023-09-30    2049
2023-10-31    3545
2023-11-30    2628
2023-12-31    2552
2024-01-31    1994
2024-02-29    1580
2024-03-31    1752
2024-04-30    1915
2024-05-31    1835
2024-06-30    1827
2024-07-31    2157
2024-08-31    1334
2024-09-30    2134
2024-10-31    3611
2024-11-30    3086
2024-12-31    2713
Freq: ME, Name: quantidade, dtype: int64

# Preparar e Inserir Pedidos

In [217]:
q = f'horario < "{datetime.now()}"'
df_ins = df.query(q).sort_values(['data', 'nota', 'produto'])

In [229]:
df_ins.head()
# df_ins[:30]
# df_ins[-30:]

,nota,data,horario,produto,quantidade
32591,20210101-3,2021-01-01,2021-01-01 10:31:00,P23,14
69596,20210101-3,2021-01-01,2021-01-01 15:39:00,P25,3
76715,20210102-1,2021-01-02,2021-01-02 09:16:00,P15,13
72375,20210102-1,2021-01-02,2021-01-02 00:10:00,P24,9
73081,20210102-3,2021-01-02,2021-01-02 11:30:00,P19,2


In [219]:
df_ins.shape

(8162, 5)

In [220]:
representantes = list(db['venda_representante'].find())

estoques = {}
for e in db['venda_estoque'].find():
  estoques[f"P{e['id']:02}"] = e

In [221]:
estoques

{'P01': {'_id': ObjectId('66449990ee56e6635d192f00'),
  'id': 1,
  'produto': {'marca': 'KazeKami',
   'categoria': 'Secadores',
   'descricao': 'Secador de Parede 1200W'},
  'quantidade': 1000,
  'preco_compra': 79.25548255817442,
  'preco_venda': 149.73},
 'P02': {'_id': ObjectId('66449990ee56e6635d192f01'),
  'id': 2,
  'produto': {'marca': 'KazeKami',
   'categoria': 'Secadores',
   'descricao': 'Secador de Parede 1200W'},
  'quantidade': 1000,
  'preco_compra': 73.9690008760712,
  'preco_venda': 169.9},
 'P03': {'_id': ObjectId('66449990ee56e6635d192f02'),
  'id': 3,
  'produto': {'marca': 'KazeKami',
   'categoria': 'Secadores',
   'descricao': 'Secador de Parede 1400W'},
  'quantidade': 1000,
  'preco_compra': 295.4559744929262,
  'preco_venda': 460.35},
 'P04': {'_id': ObjectId('66449990ee56e6635d192f03'),
  'id': 4,
  'produto': {'marca': 'KazeKami',
   'categoria': 'Secadores',
   'descricao': 'Secador de Parede 1400W (Preto)'},
  'quantidade': 1000,
  'preco_compra': 122.499

In [222]:
def gen_pedido(i, itens_pedido, total, representantes, horario):
  representante = random.choice(representantes)
  cliente = random.choice(representante['clientes'])

  etapas = [{
    'id': 1,
    'etapa': 'criado',
    'horario': horario - timedelta(seconds = random.randrange(9999, 99999)),
    'user_id': representante['id']
  }]
  
  etapa_id = 2
  if representante['id'] != 2:
    etapas.append({
      'id': etapa_id,
      'etapa': 'enviado',
      'horario': horario - timedelta(seconds = random.randrange(99, 9999)),
      'user_id': representante['id']
    })
    etapa_id += 1

  etapas.append({
    'id': etapa_id,
    'etapa': 'confirmado',
    'horario': horario,
    'user_id': 2
  })

  pedido = {
    'id': i,
    'etapa': 'confirmado',
    'etapas_pedido': etapas,
    'representante_id': representante['id'],
    'cliente': cliente,
    'horario': horario,
    'itens_pedido': itens_pedido,
    'total': round(total, 2)
  }
  return pedido


In [223]:
pedidos = []
itens_pedido = []
i = 0
nota = df_ins.iloc[0, 0]
horario = df_ins.iloc[0, 2]
prod_ant = ''
total = 0
for e in df_ins.itertuples():
  if nota != e.nota:
    i += 1
    pedido = gen_pedido(i, itens_pedido, total, representantes, horario)
    pedidos.append(pedido)
    itens_pedido = []
    prod_ant = ''
    total = 0
    nota = e.nota
    horario  = e.horario

  estoque = estoques[e.produto]
  total_prod = round(e.quantidade * estoque['preco_venda'], 2)
  total += total_prod
  if e.produto == prod_ant:
    itens_pedido[-1]['quantidade'] += e.quantidade
  else:
    item = {
      'id': estoque['id'],
      'produto': estoque['produto'],
      'preco_compra': estoque['preco_compra'],
      'preco_venda': estoque['preco_venda'],
      'quantidade': e.quantidade,
      'total': total_prod
    }
    itens_pedido.append(item)
    prod_ant = e.produto

if nota != df_ins.iloc[0, 0]:
  pedido = gen_pedido(i + 1, itens_pedido, total, representantes, horario)
  pedidos.append(pedido)

In [224]:
# from pprint import pprint
# pprint(pedidos)

In [225]:
db['venda_pedido'].delete_many({})
db['venda_pedido'].insert_many(pedidos)

InsertManyResult([ObjectId('66449991ee56e6635d192f1d'), ObjectId('66449991ee56e6635d192f1e'), ObjectId('66449991ee56e6635d192f1f'), ObjectId('66449991ee56e6635d192f20'), ObjectId('66449991ee56e6635d192f21'), ObjectId('66449991ee56e6635d192f22'), ObjectId('66449991ee56e6635d192f23'), ObjectId('66449991ee56e6635d192f24'), ObjectId('66449991ee56e6635d192f25'), ObjectId('66449991ee56e6635d192f26'), ObjectId('66449991ee56e6635d192f27'), ObjectId('66449991ee56e6635d192f28'), ObjectId('66449991ee56e6635d192f29'), ObjectId('66449991ee56e6635d192f2a'), ObjectId('66449991ee56e6635d192f2b'), ObjectId('66449991ee56e6635d192f2c'), ObjectId('66449991ee56e6635d192f2d'), ObjectId('66449991ee56e6635d192f2e'), ObjectId('66449991ee56e6635d192f2f'), ObjectId('66449991ee56e6635d192f30'), ObjectId('66449991ee56e6635d192f31'), ObjectId('66449991ee56e6635d192f32'), ObjectId('66449991ee56e6635d192f33'), ObjectId('66449991ee56e6635d192f34'), ObjectId('66449991ee56e6635d192f35'), ObjectId('66449991ee56e6635d192f

In [226]:
# atualizando controle do sequencial de pedidos
print(db['__schema__'].find_one({'name': 'venda_pedido'})['auto']['seq'])
print(len(pedidos))

db['__schema__'].update_one({'name': 'venda_pedido'}, {
  '$set': {
    'auto.seq': len(pedidos)
  }}, upsert=False)

print(db['__schema__'].find_one({'name': 'venda_pedido'}))

4882
4820
{'_id': ObjectId('6636a5166034bd29b7dcf81f'), 'name': 'venda_pedido', 'auto': {'field_names': ['id'], 'seq': 4820}, 'fields': {'cliente': {'type_code': 'object'}, 'etapa': {'type_code': 'string'}, 'etapas_pedido': {'type_code': 'array'}, 'horario': {'type_code': 'date'}, 'id': {'type_code': 'int'}, 'itens_pedido': {'type_code': 'array'}, 'representante_id': {'type_code': 'int'}, 'total': {'type_code': 'double'}}}


In [227]:
# mantendo a diferença com 1000 no estoque
for k in estoques.keys():
  estoques[k]['quant_venda'] = 0

for p in pedidos:
  for i in p['itens_pedido']:
    estoques[f'P{i["id"]:02}']['quant_venda'] += i['quantidade']

for k in estoques.keys():
  q = estoques[k]['quant_venda']

  db['venda_estoque'].update_one({'id': estoques[k]['id']}, {
    '$set': {
      'quantidade': (q // 1000 + 1) * 1000 - q
    }}, upsert=False)

In [228]:
for e in db['venda_estoque'].find():
  print(e)

{'_id': ObjectId('66449990ee56e6635d192f00'), 'id': 1, 'produto': {'marca': 'KazeKami', 'categoria': 'Secadores', 'descricao': 'Secador de Parede 1200W'}, 'quantidade': 588, 'preco_compra': 79.25548255817442, 'preco_venda': 149.73}
{'_id': ObjectId('66449990ee56e6635d192f01'), 'id': 2, 'produto': {'marca': 'KazeKami', 'categoria': 'Secadores', 'descricao': 'Secador de Parede 1200W'}, 'quantidade': 581, 'preco_compra': 73.9690008760712, 'preco_venda': 169.9}
{'_id': ObjectId('66449990ee56e6635d192f02'), 'id': 3, 'produto': {'marca': 'KazeKami', 'categoria': 'Secadores', 'descricao': 'Secador de Parede 1400W'}, 'quantidade': 462, 'preco_compra': 295.4559744929262, 'preco_venda': 460.35}
{'_id': ObjectId('66449990ee56e6635d192f03'), 'id': 4, 'produto': {'marca': 'KazeKami', 'categoria': 'Secadores', 'descricao': 'Secador de Parede 1400W (Preto)'}, 'quantidade': 555, 'preco_compra': 122.49997960731771, 'preco_venda': 469.26}
{'_id': ObjectId('66449990ee56e6635d192f04'), 'id': 5, 'produto':